# ETL

Aquí se realizarán las transformaciones necesarias para tener datasets para optimizar las consultas.

In [12]:
import pandas as pd
import numpy as np
import ast
import json

In [13]:
#Se cargan todos los datos limpios para poder hacer las transformaciones necesarias
df_games=pd.read_parquet(r'Datasets/steam_games_clean.parquet')
df_users=pd.read_parquet(r'Datasets/users_items_clean.parquet')
df_reviews=pd.read_parquet(r'Datasets/user_reviews_clean.parquet')

## Análisis de sentimiento
En esta sección antes de realizar otras transformaciones se realizara el análisis de sentimientos mediante la librería **TextBlob** para simplificar el proceso. Pero esta análisis se puede realizar con otras librerías más complejas o mas elaboradas.

In [15]:
from textblob import TextBlob

#Se crea una función para devolver el el análisis de sentimiento 
#de un review pero en un rango de 0 a 2
def sentiment(review):
    blob=TextBlob(review)
    resultado=1 + round(blob.sentiment.polarity)
    return resultado

#Se aplica la función a todos los reviews y se crea la columna nueva sentiment
df_reviews['sentiment']=df_reviews['review'].apply(sentiment)

#Se elimina la columna review
df_reviews.drop(columns=['review'],inplace=True)

## Transformación
Una vez finalizado el análisis de sentimiento vamos a comenzar a transformar y unir los datasets para que la API y el modelo de similitud del coseno puedan utilizarlos

### Dataset developers

In [16]:
#Se agrupa los datos por item id y se crea un nuevo dataframe con la cantidad de reviews positivos, negativos y neutros
df_item_sentiment_counts = df_reviews.groupby('item_id')['sentiment'].value_counts().unstack().reset_index()

#Se crea un nuevo dataframe con la cantidad de recomendaciones positivas y negativas
df_item_recommend_count=df_reviews.groupby('item_id')['recommend'].value_counts().unstack().reset_index()

#Se unen los dos dataframes anteriores
df_counts=pd.merge(df_item_sentiment_counts,df_item_recommend_count,on='item_id')

#Se rellenan los valores nulos con 0
df_counts.fillna(0,inplace=True)

#Se une el dataframe de los juegos con el dataframe de los conteos
df_developers=pd.merge(df_games[['price','developer','release_year','id']],df_counts,left_on='id',right_on='item_id')

#Se elimina la columna id que no se van a utilizar
df_developers.drop('id',axis=1,inplace=True)

#Se renombran las columnas
df_developers.rename(columns={'0_x':'Negative','1_x':'Neutral',2:'Positive','False_y':'False','True_y':'True'},inplace=True)

#Se cambian los tipos de datos de las columnas
df_developers = df_developers.astype({'release_year':'int16','Negative': 'int16', 'Neutral': 'int16', 'Positive': 'int16', 'False': 'int16', 'True': 'int16'})

In [17]:
#Corroboramos el dafatrame final
df_developers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2971 entries, 0 to 2970
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         2971 non-null   float64
 1   developer     2971 non-null   object 
 2   release_year  2971 non-null   int16  
 3   item_id       2971 non-null   int32  
 4   Negative      2971 non-null   int16  
 5   Neutral       2971 non-null   int16  
 6   Positive      2971 non-null   int16  
 7   False         2971 non-null   int16  
 8   True          2971 non-null   int16  
dtypes: float64(1), int16(6), int32(1), object(1)
memory usage: 93.0+ KB


In [18]:
#Se guarda el dataframe en un archivo parquet
df_developers.to_parquet(r'Datasets/developers.parquet')

### Datasets de users

In [22]:
df_games=pd.read_parquet(r'Datasets/steam_games_clean.parquet')
df_users=pd.read_parquet(r'Datasets/users_items_clean.parquet')
df_reviews=pd.read_parquet(r'Datasets/user_reviews_clean.parquet')

In [23]:
#Se cambia el tipo de dato de los usuarios a string
df_users['user_id']=df_users['user_id'].astype('str')

In [24]:
#Se crea el df1 de la unión de los juegos y los usuarios y se eliminan las columnas que no se utilizan
df1=pd.merge(df_games,df_users,left_on='id',right_on='item_id',how= 'inner')
df1.drop(columns=['id','developer','app_name','tags'],inplace=True)
df1.drop(columns=['item_id'],inplace=True)

#En la columna de generos se toma solo el primer genero como más importante
df1['genres'] = df1['genres'].apply(lambda x: x[0] if len(x) > 0 else None)

#Se crea un nuevo dataframe con la cantidad de items por usuario
df_grouped = df1.groupby(['release_year', 'items_count', 'user_id','genres']).sum().reset_index()


In [26]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1219006 entries, 0 to 1219005
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   release_year      1219006 non-null  object 
 1   items_count       1219006 non-null  int16  
 2   user_id           1219006 non-null  object 
 3   genres            1219006 non-null  object 
 4   price             1219006 non-null  float64
 5   playtime_forever  1219006 non-null  float32
dtypes: float32(1), float64(1), int16(1), object(3)
memory usage: 44.2+ MB


In [27]:
df_grouped[df_grouped['user_id']=='76561198064710856']

,release_year,items_count,user_id,genres,price,playtime_forever
3,1983,43,76561198064710856,Action,9.99,37.0
101,1984,43,76561198064710856,Action,9.99,92.0
367606,2009,43,76561198064710856,Action,0.00,21.0
541589,2011,43,76561198064710856,Action,9.99,0.0
754098,2013,43,76561198064710856,Action,14.99,109.0
878543,2014,43,76561198064710856,Action,1.99,178.0
999554,2015,43,76561198064710856,Action,14.99,497.0
999555,2015,43,76561198064710856,Free to Play,0.00,1.0
1110294,2016,43,76561198064710856,Action,0.00,12.0
1189323,2017,43,76561198064710856,Action,60.00,0.0


In [28]:
#Se guarda el dataframe en un archivo parquet
df_grouped.to_parquet(r'Datasets/users.parquet')

### Dataset users_recommends

In [31]:
#Se crea el dataframe 2 con los reviews y se eliminan las columnas que no se utilizan
df2=df_reviews.copy()
df2=df2.drop(columns=(['review','item_id']),axis=1)
#Se agrupan por user id y se cuentan las recomendaciones 
total=df2.groupby('user_id')['recommend'].count()
#Se cuentan las recomendaciones positivas
positivos=df2[df2['recommend']==True].groupby('user_id')['recommend'].count()
positivos=positivos.reindex(total.index,fill_value=0)
#Se calcula el porcentaje
porcentaje_positivos=positivos*100/total
#Se unen los dataframes
df2=pd.merge(df2,porcentaje_positivos,on='user_id')
#Se eliminan los duplicados
df2.drop_duplicates(inplace=True)
#Se eliminan la columna que no se utiliza
df2.drop(columns=['recommend_x'],inplace=True)
#Se renombran la columna
df2.rename(columns={'recommend_y':'perc_recomm'},inplace=True)
#Se convierte al columna a string y se le agrega el signo de porcentaje
df2['perc_recomm']=df2['perc_recomm'].astype('str')+'%'
#Se convierte la columna user_id a string
df2['user_id']=df2['user_id'].astype('str')

In [32]:
#Se guarda el dataframe en un archivo parquet
df2.to_parquet(r'Datasets/user_recommends.parquet')

### Dataset data_ML

In [34]:
df=df_games.copy()

#Se eliminan las columnas que no se van a utilizar
df.drop(columns=['developer','price'],inplace=True)

#Para simplicar se elige el primer valor de la lista dentro de la columna genres
df['genres'] = df['genres'].apply(lambda x: x[0] if len(x) > 0 else None)

#Se toma solo el primer tag para simplificar
df['tags'] = df['tags'].apply(lambda x: x[0] if len(x) > 0 else None)

#Se convierte la columna del año a entero
df_developers['release_year']=df_developers['release_year'].astype('int16')

In [35]:
df.to_parquet(r'Datasets/data_ML.parquet')

# Prueba de ENDPOINTS

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. 

In [45]:
#Se carga el dataset de los desarrolladores
df_developers=pd.read_parquet(r'Datasets/developers.parquet')

#Se normaliza la entrada
developer='valve'
developer=developer.title()

#Se verifica que el desarrollador este en el dataset
if developer in df_developers['developer'].values:
    df_developers=df_developers[df_developers['developer']==developer]
    #Se eliminan las columnas que no se van a utilizar
    df_developers.drop(columns=['Negative','Neutral','Positive','True','False','developer'],inplace=True)
    #Total de items por cada año
    total = df_developers.groupby('release_year')['price'].count()
    #Cuenta la cantidad de items que no son gratis
    no_ceros = df_developers[df_developers['price'] != 0].groupby('release_year')['price'].count()
    no_ceros= no_ceros.reindex(total.index, fill_value=0)
    #Calacula la proporción de items gratis
    proporcion_gratis =round((1- no_ceros / total)*100,2)
    proporcion_gratis=proporcion_gratis.astype('str')+'%'
    #Doy formato a la respuesta
    data = [{'Año': year, 'Cantidad de items': total[year], 'Contenido free': proporcion_gratis[year]} for year in total.index]
    data={developer:data}  
else:
    print('No existe el desarrollador')

In [46]:
data

{'Valve': [{'Año': 1998, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 1999, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2000, 'Cantidad de items': 2, 'Contenido free': '0.0%'},
  {'Año': 2001, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2003, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2004, 'Cantidad de items': 5, 'Contenido free': '0.0%'},
  {'Año': 2006, 'Cantidad de items': 2, 'Contenido free': '0.0%'},
  {'Año': 2007, 'Cantidad de items': 3, 'Contenido free': '33.33%'},
  {'Año': 2008, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2009, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2010, 'Cantidad de items': 2, 'Contenido free': '50.0%'},
  {'Año': 2011, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2012, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2016, 'Cantidad de items': 1, 'Contenido free': '100.0%'}]}

In [47]:
data

{'Valve': [{'Año': 1998, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 1999, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2000, 'Cantidad de items': 2, 'Contenido free': '0.0%'},
  {'Año': 2001, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2003, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2004, 'Cantidad de items': 5, 'Contenido free': '0.0%'},
  {'Año': 2006, 'Cantidad de items': 2, 'Contenido free': '0.0%'},
  {'Año': 2007, 'Cantidad de items': 3, 'Contenido free': '33.33%'},
  {'Año': 2008, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2009, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2010, 'Cantidad de items': 2, 'Contenido free': '50.0%'},
  {'Año': 2011, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2012, 'Cantidad de items': 1, 'Contenido free': '0.0%'},
  {'Año': 2016, 'Cantidad de items': 1, 'Contenido free': '100.0%'}]}

In [48]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [49]:
json.dumps(data, cls=NpEncoder, ensure_ascii=False)

'{"Valve": [{"Año": 1998, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 1999, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2000, "Cantidad de items": 2, "Contenido free": "0.0%"}, {"Año": 2001, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2003, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2004, "Cantidad de items": 5, "Contenido free": "0.0%"}, {"Año": 2006, "Cantidad de items": 2, "Contenido free": "0.0%"}, {"Año": 2007, "Cantidad de items": 3, "Contenido free": "33.33%"}, {"Año": 2008, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2009, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2010, "Cantidad de items": 2, "Contenido free": "50.0%"}, {"Año": 2011, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2012, "Cantidad de items": 1, "Contenido free": "0.0%"}, {"Año": 2016, "Cantidad de items": 1, "Contenido free": "100.0%"}]}'

def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [193]:
df_recc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29142 entries, 0 to 59302
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      29142 non-null  object
 1   perc_recomm  29142 non-null  object
dtypes: object(2)
memory usage: 683.0+ KB


In [219]:
user_id='maplemage'
df_users=pd.read_parquet(r'Datasets/users.parquet')
df_recc=pd.read_parquet(r'Datasets/user_recommends.parquet')
if user_id in df_users['user_id'].values:
    df_users=df_users[df_users['user_id']==user_id]
    cantidad_juegos=df_users['items_count'].values[0]
    dinero_gastado=df_users['price'].sum().round()
    if user_id in df_recc['user_id'].values:
        recomendaciones=df_recc[df_recc['user_id']==user_id]['perc_recomm']
    else:
        recomendaciones=0
    respuesta={'Dinero gastado':dinero_gastado,'Recomendaciones':recomendaciones,'Cantidad de juegos':cantidad_juegos,}
    print(respuesta)
else:
    print('No existe el usuario')

{'Dinero gastado': 6004.0, 'Recomendaciones': 18    100.0%
Name: perc_recomm, dtype: object, 'Cantidad de juegos': 629}


def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [256]:
genre='Action'
df_users=pd.read_parquet(r'Datasets/users.parquet')
if genre in df_users['genres'].values:
    df_users=df_users[df_users['genres']==genre]
    df_users.drop(columns=['items_count','price','genres'],inplace=True)
    usuario=df_users.groupby('user_id')['playtime_forever'].sum().idxmax()
    df_users=df_users[df_users['user_id']==usuario]
    horas_jugadas = [{'Año': row['release_year'], 'Horas': row['playtime_forever']} for index, row in df_users.iterrows()]
    respuesta={'Usuario con mas horas jugadas para el genero '+ genre:usuario,'Horas jugadas':horas_jugadas}
else:
    print('No existe el genero')

def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [61]:
#Se carga el dataset de los desarrolladores
df_developers=pd.read_parquet(r'Datasets/developers.parquet')
anio=2011
if anio in df_developers['release_year'].values:
    df_developers=df_developers[df_developers['release_year']==anio]
    df_developers.drop(columns=['price','item_id','Negative','Neutral','Positive','False','release_year'],axis=1,inplace=True)
    df_developers=df_developers.groupby('developer')['True'].sum()
    df_developers=df_developers.sort_values(ascending=False)
    respuesta=[{'Puesto '+str(i+1):df_developers.index[i]} for i in range(3)]
    json.dumps(respuesta, cls=NpEncoder, ensure_ascii=False)
else:
    print('Año no encontrado')

In [62]:
df=pd.read_parquet(r'Datasets/developers.parquet')

In [69]:
df['release_year'].value_counts()

release_year
2015    668
2014    550
2016    428
2013    331
2012    233
2011    160
2010    109
2009     92
2008     63
2007     55
2017     50
2006     47
2005     30
2003     27
2004     25
2001     16
1998     14
2002     14
1997     13
1999     11
2000      9
1994      6
1995      5
1996      5
1993      3
1992      3
1990      2
1989      1
1991      1
Name: count, dtype: int64

def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [51]:
#Se carga el dataset de los desarrolladores
df_developers=pd.read_parquet(r'Datasets/developers.parquet')
desarrollador="valve"
desarrollador=desarrollador.title()
desarrollador

if desarrollador in df_developers['developer'].values:
    respuesta=df_developers[df_developers['developer']==desarrollador][['Positive','Negative']].sum()
else:
    print('Desarrollador no encontrado')

In [52]:
respuesta = [f"{k} = {v}" for k, v in respuesta.items()]

In [53]:
json.dumps(respuesta, cls=NpEncoder, ensure_ascii=False)

'["Positive = 1009", "Negative = 113"]'